In [9]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold

from sklearn.model_selection import KFold
from sklearn import metrics

In [2]:
data = pd.read_csv(r"data_no_corr_95.csv")

data_defects = data.loc[data['defects'] == True]

data_no_defects = data.loc[data['defects'] == False]
data_no_defects = data_no_defects.sample(n=len(data_defects))

print(len(data_no_defects))
print(len(data_defects))

balanced_data = data_defects.append(data_no_defects)
balanced_data = balanced_data.sample(frac = 1).reset_index(drop = True)
balanced_data

2665
2665


,lineCount,cyclomaticComplexity,essentialComplexity,designComplexity,operatorsOperands,volume,programLength,difficulty,intelligence,effort,...,hlineCount,hlineComments,hlineBlanks,hCodeAndComment,uniqueOperators,uniqueOperands,totalOperators,totalOperands,branchCount,defects
0,14.0,1.0,1.0,1.0,34.0,149.34,0.13,7.73,19.33,1153.98,...,8,0,4,0,10.0,11.0,17.0,17.0,1.0,False
1,105.0,6.0,1.0,4.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0,0,0,0,0.0,0.0,0.0,0.0,11.0,True
2,58.0,10.0,5.0,1.0,187.0,1038.71,0.03,33.52,30.99,34816.70,...,6,6,9,0,21.0,26.0,104.0,83.0,19.0,True
3,18.0,2.0,1.0,2.0,33.0,132.00,0.10,10.00,13.20,1320.00,...,12,1,3,0,10.0,6.0,21.0,12.0,3.0,True
4,92.0,5.0,3.0,4.0,418.0,2726.85,0.05,19.68,138.59,53651.64,...,84,0,6,0,15.0,77.0,216.0,202.0,9.0,True
5,48.0,9.0,8.0,6.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0,0,0,0,0.0,0.0,0.0,0.0,17.0,True
6,31.0,7.0,5.0,5.0,113.0,616.92,0.04,22.50,27.42,13880.60,...,28,0,1,0,22.0,22.0,68.0,45.0,13.0,True
7,55.0,11.0,5.0,8.0,217.0,1311.63,0.04,26.83,48.89,35190.17,...,0,8,17,0,25.0,41.0,129.0,88.0,21.0,False
8,5.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,...,3,3,2,0,4.0,1.0,4.0,1.0,1.0,False
9,25.0,5.0,1.0,4.0,105.0,538.57,0.09,11.69,46.08,6294.59,...,21,0,2,0,11.0,24.0,54.0,51.0,9.0,False


In [8]:
features = list(balanced_data.columns.values)
features.remove('defects')

In [15]:
train = balanced_data.sample(frac = 0.8, random_state = 1)
test = balanced_data.loc[~balanced_data.index.isin(train.index)]

train_data_array=train.as_matrix(columns = features)
train_class_array= train['defects'].values
test_data_array=test.as_matrix(columns = features)
test_class_array= test['defects'].values

In [16]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

kf=KFold(n_splits=10, random_state=None, shuffle=False)
validation_score=0
for train_index, test_index in kf.split(train_data_array):
    X_train, X_test = train_data_array[train_index], train_data_array[test_index]
    y_train, y_test = train_class_array[train_index], train_class_array[test_index]
    clf.fit(X_train, y_train)
    validation_pred=clf.predict(X_test)
    validation_score+=metrics.accuracy_score(y_test,validation_pred)

print("Validation Accuracy:   %0.3f" % (validation_score/kf.get_n_splits()))

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

Validation Accuracy:   0.518
accuracy:   0.517
Precision:   0.727
Recall:   0.060
F-measure:   0.111


In [17]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

clf.fit(train_data_array, train_class_array)

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

accuracy:   0.680
Precision:   0.671
Recall:   0.712
F-measure:   0.691


In [18]:
from sklearn import svm

clf = svm.SVC()

clf.fit(train_data_array, train_class_array)

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

accuracy:   0.612
Precision:   0.570
Recall:   0.920
F-measure:   0.704
